# ANALYSING NEGATIVE REVIEWS ON AIRBNB LISTING USING SENTIMENT ANALYSIS
## Table of Contents
<ul>
<li><a href="#intro">Introduction</a></li>
<li><a href="#wrangling">Data Preprocessing and Wrangling</a></li>
 <li><a href="#sal">Sentiment Analysis</a></li>
<li><a href="#pred">Predictive Analysis </a></li>
</ul>

<a id='intro'></a>
## Section I: Introduction

**Datasets**:

[Air-bnb-reviews-top 50 cities](https://drive.google.com/file/d/14-NIljR74Imltim1NRicvAG3QylzaiQa/view?usp=drivesdk)

**Description**: This dataset collects information about various reviews by different users of airbnb in the top 50 cites in the US with every detail associated with the data

[Air-bnb-users-top 50 cities](https://drive.google.com/file/d/1e5JpzUd5TkFhddqgy2IYMl0a6MkCZfkX/view?usp=drivesdk)

**Description**: This dataset collects information about users across the top 50 users of Air BnB int the US.

[Air-bnb-listings-top 50 cities](https://drive.google.com/file/d/1yN7p3TLJM4ws-xUcfh8H6F46Au9O7nfB/view?usp=drivesdk)

**Description**: This dataset collects information about various listings of Airbnb

[Data Dictionary](https://docs.google.com/spreadsheets/d/1iWCNJcSutYqpULSQHlNyGInUvHg2BoUGoNRIGa6Szc4/edit?usp=sharing)

The Data dictionary describes all the features in the data set.


# Problem Statement:

Who are those giving negative reviews on Airbnb?
RESEARCH QUESTIONS

1. Are they newcomers or established users?

2. Are they experienced or inexperienced users?

3. Which age/gender/ethnicity?

4. What's the trend over time?

5. What is the distribution of the location? Are they the same in Western and non-Western countries (Are they the same within the USA).

<a id='wrangling'></a>
## Section II: Data Preprocessing and Wrangling

1. Gathering the Data
2. Access the Data 
3. Clean the Data

**Gather**

In [ ]:
#importing the required libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import tensorflow.keras as keras
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download([
    "names",
    "stopwords",
   "averaged_perceptron_tagger",
    "vader_lexicon",
 "punkt",
 ])
import re
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.tokenize import RegexpTokenizer
import nltk
from nltk.stem import WordNetLemmatizer
nltk.download('averaged_perceptron_tagger')
from nltk.corpus import wordnet
 
lemmatizer = WordNetLemmatizer()
%matplotlib inline

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#!gunzip "/content/drive/MyDrive/Datasets/airbnb_listing_reviews_top50_cities.tar.gz" 

gzip: /content/drive/MyDrive/Datasets/airbnb_listing_reviews_top50_cities.tar.gz: No such file or directory


In [ ]:
#!gunzip "/content/drive/MyDrive/Datasets/Copy of airbnb_users_top50_cities.tar.gz" -c '/content/sample_data/users'

gzip: /content/drive/MyDrive/Datasets/Copy of airbnb_users_top50_cities.tar.gz: No such file or directory
gzip: /content/sample_data/users.gz: No such file or directory


In [ ]:
#!gzip --help

Usage: gzip [OPTION]... [FILE]...
Compress or uncompress FILEs (by default, compress FILES in-place).

Mandatory arguments to long options are mandatory for short options too.

  -c, --stdout      write on standard output, keep original files unchanged
  -d, --decompress  decompress
  -f, --force       force overwrite of output file and compress links
  -h, --help        give this help
  -k, --keep        keep (don't delete) input files
  -l, --list        list compressed file contents
  -L, --license     display software license
  -n, --no-name     do not save or restore the original name and time stamp
  -N, --name        save or restore the original name and time stamp
  -q, --quiet       suppress all warnings
  -r, --recursive   operate recursively on directories
  -S, --suffix=SUF  use suffix SUF on compressed files
  -t, --test        test compressed file integrity
  -v, --verbose     verbose mode
  -V, --version     display version number
  -1, --fast        compress faster
  -9

In [ ]:
#loading the datasets
df_reviews = pd.read_csv( '/content/drive/MyDrive/Datasets/reviews',  delimiter ='\t')
df_users = pd.read_csv('/content/drive/MyDrive/Datasets/Copy of airbnb_users_top50_cities.tar',delimiter ='\t')
#df_listings = pd.read_csv('airbnb_listings_top50_cities.tsv',delimiter ='\t')

**Access**

In [ ]:
#check info of the review dataset
df_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7416669 entries, 0 to 7416668
Data columns (total 15 columns):
 #   Column                     Dtype 
---  ------                     ----- 
 0   review_id                  int64 
 1   createdAt                  object
 2   localizedDate              object
 3   reviewee_id                int64 
 4   recommendedNumberOfLines   object
 5   roomTypeListingTitle       object
 6   language                   object
 7   highlightReviewMentioned   object
 8   localizedReview            object
 9   highlightedReviewSentence  object
 10  response                   object
 11  reviewer_id                int64 
 12  rating                     int64 
 13  comments                   object
 14  listing_id                 int64 
dtypes: int64(5), object(10)
memory usage: 848.8+ MB


In [ ]:
#view first 5 columns
df_reviews.head()

,review_id,createdAt,localizedDate,reviewee_id,recommendedNumberOfLines,roomTypeListingTitle,language,highlightReviewMentioned,localizedReview,highlightedReviewSentence,response,reviewer_id,rating,comments,listing_id
0,66,2008-08-29T00:27:05Z,August 2008,990,None,None,en,None,None,None,None,1219,5,Alexandra was a great hostess. She went out of...,686
1,70,2008-08-29T00:27:06Z,August 2008,990,None,None,en,None,None,None,None,1257,5,I enjoyed my stay with Alexandra. She was more...,686
2,74,2008-08-29T00:27:06Z,August 2008,990,None,None,en,None,None,None,None,1398,5,Alexandra was so kind and helpful. She opened...,686
3,414,2009-01-12T16:27:07Z,January 2009,3461,None,None,en,None,None,None,None,5639,0,"Cute place, good location, great price! TV onl...",3054
4,749,2009-02-21T17:48:32Z,February 2009,933,None,None,en,None,None,None,None,6777,5,I really enjoyed my stay at Jill's. It's a rea...,592


In [ ]:
#view descriptions
df_reviews.describe(include ='all')

,review_id,createdAt,localizedDate,reviewee_id,recommendedNumberOfLines,roomTypeListingTitle,language,highlightReviewMentioned,localizedReview,highlightedReviewSentence,response,reviewer_id,rating,comments,listing_id
count,7.416669e+06,7416669,7416669,7.416669e+06,7416669,7416669,7416669,7416669,7416669,7416669,7415887,7.416669e+06,7.416669e+06,7414749,7.416669e+06
unique,NaN,6533682,160,NaN,1,1,119,1,1,1,630880,NaN,NaN,6973141,NaN
top,NaN,2019-08-18T23:40:11Z,October 2021,NaN,None,None,en,None,None,None,None,NaN,NaN,Great place,NaN
freq,NaN,16,248281,NaN,7416669,7416669,7250468,7416669,7416669,7416669,6704733,NaN,NaN,11613,NaN
mean,1.268992e+17,NaN,NaN,9.443390e+07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.442163e+08,4.797307e+00,NaN,1.022600e+15
std,2.139028e+17,NaN,NaN,1.050638e+08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.225587e+08,6.058997e-01,NaN,2.364288e+16
min,6.600000e+01,NaN,NaN,2.300000e+01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000e+00,0.000000e+00,NaN,1.770000e+02
25%,3.336374e+08,NaN,NaN,1.397648e+07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.918144e+07,5.000000e+00,NaN,1.285109e+07
50%,6.080838e+08,NaN,NaN,5.007276e+07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.119586e+08,5.000000e+00,NaN,2.304326e+07
75%,3.956174e+17,NaN,NaN,1.439301e+08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.258990e+08,5.000000e+00,NaN,3.777384e+07


In [ ]:
#check for missing values
df_reviews.isna().sum()

review_id                       0
createdAt                       0
localizedDate                   0
reviewee_id                     0
recommendedNumberOfLines        0
roomTypeListingTitle            0
language                        0
highlightReviewMentioned        0
localizedReview                 0
highlightedReviewSentence       0
response                      782
reviewer_id                     0
rating                          0
comments                     1920
listing_id                      0
dtype: int64

In [ ]:
#check for basic info of the user database
df_users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5697210 entries, 0 to 5697209
Data columns (total 24 columns):
 #   Column                Dtype  
---  ------                -----  
 0   user_id               int64  
 1   first_name            object 
 2   last_name             float64
 3   is_superhost          object 
 4   recommendation_count  float64
 5   has_profile_pic       object 
 6   total_listings_count  float64
 7   location              object 
 8   friends_count         float64
 9   response_time         object 
 10  reviewee_count        float64
 11  verification_labels   object 
 12  listings_count        float64
 13  neighborhood          object 
 14  created_at            object 
 15  identity_verified     object 
 16  picture_url           object 
 17  acceptance_rate       object 
 18  verifications         object 
 19  thumbnail_url         object 
 20  about                 object 
 21  response_rate         object 
 22  groups                float64
 23  wishlis

In [ ]:
#check the first 5 columns
df_users.head()

,user_id,first_name,last_name,is_superhost,recommendation_count,has_profile_pic,total_listings_count,location,friends_count,response_time,...,created_at,identity_verified,picture_url,acceptance_rate,verifications,thumbnail_url,about,response_rate,groups,wishlists_count
0,1,Nathan,NaN,False,0.0,True,0.0,"San Francisco, California, United States",0.0,NaN,...,2008-03-02T06:20:00Z,True,https://a0.muscache.com/im/users/1/profile_pic...,NaN,"['email', 'phone', 'facebook', 'reviews', 'jum...",https://a0.muscache.com/im/users/1/profile_pic...,I live with my wife who is a doctor at Stanfor...,NaN,NaN,10.0
1,2,Joe,NaN,False,5.0,True,1.0,"San Francisco, California, United States",9.0,NaN,...,2008-03-02T06:20:00Z,True,https://a0.muscache.com/im/pictures/user/7509e...,NaN,"['email', 'phone', 'manual_online', 'reviews',...",https://a0.muscache.com/im/pictures/user/7509e...,I'm a designer and athlete who has a knack for...,NaN,NaN,29.0
2,3,Brian,NaN,False,13.0,True,0.0,"San Francisco, California, United States",14.0,NaN,...,2008-03-02T06:20:00Z,True,https://a0.muscache.com/im/pictures/user/858bb...,NaN,"['email', 'phone', 'facebook', 'reviews', 'jum...",https://a0.muscache.com/im/pictures/user/858bb...,"I am originally from NY, and have been living ...",NaN,NaN,88.0
3,14,Janet,NaN,False,0.0,True,0.0,"Irving, Texas, United States",0.0,NaN,...,2008-03-03T14:59:16Z,True,https://a0.muscache.com/im/pictures/user/072a8...,NaN,"['email', 'phone', 'reviews', 'jumio', 'offlin...",https://a0.muscache.com/im/pictures/user/072a8...,"I enjoy traveling, cooking and playing scrabble.",NaN,NaN,130.0
4,15,Kat,NaN,False,1.0,True,0.0,"Silver Spring, Maryland, United States",1.0,NaN,...,2008-03-03T15:30:32Z,True,https://a0.muscache.com/im/pictures/user/b0dcf...,NaN,"['email', 'phone', 'reviews']",https://a0.muscache.com/im/pictures/user/b0dcf...,I am one of the very first guests of Airbnb! I...,NaN,NaN,14.0


In [ ]:
df_users.describe(include='all')

,user_id,first_name,last_name,is_superhost,recommendation_count,has_profile_pic,total_listings_count,location,friends_count,response_time,...,created_at,identity_verified,picture_url,acceptance_rate,verifications,thumbnail_url,about,response_rate,groups,wishlists_count
count,5.697210e+06,5664284,0.0,5664297,5.664297e+06,5664297,5.664297e+06,5664294,122543.000000,143796,...,5664297,5664297,5664297,165622,5664297,5664297,1970948,143796,0.0,5.664297e+06
unique,NaN,384341,NaN,2,NaN,2,NaN,88579,NaN,4,...,5586551,2,5460463,101,2548,5460463,1878594,90,NaN,NaN
top,NaN,Michael,NaN,False,NaN,True,NaN,"New York, New York, United States",NaN,within an hour,...,2019-02-04T18:42:53Z,True,https://a0.muscache.com/defaults/user_pic-225x...,100%,"['email', 'phone']",https://a0.muscache.com/defaults/user_pic-50x5...,,100%,NaN,NaN
freq,NaN,47425,NaN,5575236,NaN,5460461,NaN,129489,NaN,106670,...,4,4502061,203813,66396,1901027,203813,5244,123894,NaN,NaN
mean,1.521904e+08,NaN,NaN,NaN,1.167735e-02,NaN,8.218337e-02,NaN,0.100928,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.678221e+00
std,1.247929e+08,NaN,NaN,NaN,2.017226e-01,NaN,2.987952e+00,NaN,0.677332,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.457448e+01
min,1.000000e+00,NaN,NaN,NaN,0.000000e+00,NaN,0.000000e+00,NaN,0.000000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000e+00
25%,4.462789e+07,NaN,NaN,NaN,0.000000e+00,NaN,0.000000e+00,NaN,0.000000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000e+00
50%,1.213949e+08,NaN,NaN,NaN,0.000000e+00,NaN,0.000000e+00,NaN,0.000000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.000000e+00
75%,2.382752e+08,NaN,NaN,NaN,0.000000e+00,NaN,0.000000e+00,NaN,0.000000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000e+01


In [ ]:
#check the missing values
df_users.isna().sum()

user_id                       0
first_name                32926
last_name               5697210
is_superhost              32913
recommendation_count      32913
has_profile_pic           32913
total_listings_count      32913
location                  32916
friends_count           5574667
response_time           5553414
reviewee_count            32913
verification_labels       32913
listings_count            32913
neighborhood            5614968
created_at                32913
identity_verified         32913
picture_url               32913
acceptance_rate         5531588
verifications             32913
thumbnail_url             32913
about                   3726262
response_rate           5553414
groups                  5697210
wishlists_count           32913
dtype: int64

In [ ]:
df_reviews['comments'].head(15)

0     Alexandra was a great hostess. She went out of...
1     I enjoyed my stay with Alexandra. She was more...
2     Alexandra was so kind and helpful.  She opened...
3     Cute place, good location, great price! TV onl...
4     I really enjoyed my stay at Jill's. It's a rea...
5          Very nice place to stay + the perfect host !
6     Sylvia is a hostess who is gracious and helpfu...
7     Highly recommended. Sylvia was extremely helpf...
8     Terry's Hotel Alterntv in Boston was a perfect...
9     A great place to stay in a great city. Sylvia ...
10    Easy access to downtown via RTD; lighthearted ...
11    Highly recommended! \r<br/>Cute and cozy guest...
12    A great place to stay.  Jill is really wonderf...
13    What a great little apartment!  It was clean, ...
14    Sylvia was great; "ditto" to all the previous ...
Name: comments, dtype: object

In [ ]:
df_reviews['language'].value_counts()

en          7250468
es            66085
fr            30914
de            15581
zh-CN          7922
             ...   
tk                1
mn                1
ka                1
jw                1
en,ja,id          1
Name: language, Length: 119, dtype: int64

Issues

Reviews Dataset
1. The Review data set has some missing values
2. Some of the reviews are not in english
3. Some of the reviews may be automated
4. Prescence of Stopwords, names and making the text lower case
5. Improper data types
6. Rename columns
7. Preprocess comment for analysis

Users Dataset
1. Too many columns, drop columns not needed for analysis
2. Drop missing values
3. Check the Picture Urls and validate
4. Clean the Location data



General

1. Merge the Two dataset with the User Ids
2. Select the data needed for analysis

**Clean**

In [ ]:
#create a copy of the datasets
df_rev= df_reviews.copy()
df_us = df_users.copy()

In [ ]:
#drop missing values
df_rev.dropna(inplace=True)

In [ ]:
#drop reviews not in english
df_rev = df_rev[df_rev['language'] == 'en']
df_rev.shape

(7248149, 15)

In [ ]:
#drop automated posts 
def autopost(x):
    if 'an automated posting' in x:
        return 'yes'
    else:
        return 'no'
df_rev['auto_posting'] = df_rev['comments'].apply(autopost)

In [ ]:
df_rev = df_rev[df_rev['auto_posting']=='no']
df_rev.shape

(7233405, 16)

In [ ]:
df_us.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5697210 entries, 0 to 5697209
Data columns (total 24 columns):
 #   Column                Dtype  
---  ------                -----  
 0   user_id               int64  
 1   first_name            object 
 2   last_name             float64
 3   is_superhost          object 
 4   recommendation_count  float64
 5   has_profile_pic       object 
 6   total_listings_count  float64
 7   location              object 
 8   friends_count         float64
 9   response_time         object 
 10  reviewee_count        float64
 11  verification_labels   object 
 12  listings_count        float64
 13  neighborhood          object 
 14  created_at            object 
 15  identity_verified     object 
 16  picture_url           object 
 17  acceptance_rate       object 
 18  verifications         object 
 19  thumbnail_url         object 
 20  about                 object 
 21  response_rate         object 
 22  groups                float64
 23  wishlis

In [ ]:
#Selecting Needed Columns
df_us = df_us[['user_id','first_name','is_superhost','recommendation_count','has_profile_pic','total_listings_count',
              'location', 'reviewee_count','verification_labels','listings_count','created_at','identity_verified',
              'picture_url']]

In [ ]:

df_us.isna().sum()

user_id                     0
first_name              32926
is_superhost            32913
recommendation_count    32913
has_profile_pic         32913
total_listings_count    32913
location                32916
reviewee_count          32913
verification_labels     32913
listings_count          32913
created_at              32913
identity_verified       32913
picture_url             32913
dtype: int64

In [ ]:
#Drop Missing Values
df_us.dropna(inplace=True)

In [ ]:
df_us.head()

,user_id,first_name,is_superhost,recommendation_count,has_profile_pic,total_listings_count,location,reviewee_count,verification_labels,listings_count,created_at,identity_verified,picture_url
0,1,Nathan,False,0.0,True,0.0,"San Francisco, California, United States",156.0,"['Email address', 'Phone number', 'Facebook', ...",0.0,2008-03-02T06:20:00Z,True,https://a0.muscache.com/im/users/1/profile_pic...
1,2,Joe,False,5.0,True,1.0,"San Francisco, California, United States",294.0,"['Email address', 'Phone number', 'Personal in...",1.0,2008-03-02T06:20:00Z,True,https://a0.muscache.com/im/pictures/user/7509e...
2,3,Brian,False,13.0,True,0.0,"San Francisco, California, United States",175.0,"['Email address', 'Phone number', 'Facebook', ...",0.0,2008-03-02T06:20:00Z,True,https://a0.muscache.com/im/pictures/user/858bb...
3,14,Janet,False,0.0,True,0.0,"Irving, Texas, United States",19.0,"['Email address', 'Phone number', 'Reviewed', ...",0.0,2008-03-03T14:59:16Z,True,https://a0.muscache.com/im/pictures/user/072a8...
4,15,Kat,False,1.0,True,0.0,"Silver Spring, Maryland, United States",6.0,"['Email address', 'Phone number', 'Reviewed']",0.0,2008-03-03T15:30:32Z,True,https://a0.muscache.com/im/pictures/user/b0dcf...


In [ ]:
#Drop users without profile pictures
df_us.has_profile_pic.value_counts()

True     5460445
False     203836
Name: has_profile_pic, dtype: int64

In [ ]:
#Drop users without profile pictures
df_us = df_us[df_us['has_profile_pic']== True]

In [ ]:
df_us.shape

(5460445, 13)

In [ ]:
#validate the url
from urllib.parse import urlparse

def uri_validator(x):
  try:
    result = urlparse(x)
    return all([result.scheme, result.netloc])
  except:
    return False




In [ ]:
#validate the url
df_us['picture_url_status'] = df_us['picture_url'].apply(uri_validator)

In [ ]:
df_us.picture_url_status.value_counts()

True    5460445
Name: picture_url_status, dtype: int64

In [ ]:
#select valid url
df_us = df_us[df_us['picture_url_status']== True]

[nltk_data] Downloading package names to /root/nltk_data...
[nltk_data]   Unzipping corpora/names.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
# set stopword

nltk.download('names')
wn = nltk.download('wordnet')

stop_words = set(stopwords.words('english'))



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package names to /root/nltk_data...
[nltk_data]   Package names is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [ ]:
# select a random sample of 1 million records
df_rev =df_rev.sample(n=1024000)

In [ ]:
df_rev.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1024000 entries, 4407959 to 3260746
Data columns (total 16 columns):
 #   Column                     Non-Null Count    Dtype 
---  ------                     --------------    ----- 
 0   review_id                  1024000 non-null  int64 
 1   createdAt                  1024000 non-null  object
 2   localizedDate              1024000 non-null  object
 3   reviewee_id                1024000 non-null  int64 
 4   recommendedNumberOfLines   1024000 non-null  object
 5   roomTypeListingTitle       1024000 non-null  object
 6   language                   1024000 non-null  object
 7   highlightReviewMentioned   1024000 non-null  object
 8   localizedReview            1024000 non-null  object
 9   highlightedReviewSentence  1024000 non-null  object
 10  response                   1024000 non-null  object
 11  reviewer_id                1024000 non-null  int64 
 12  rating                     1024000 non-null  int64 
 13  comments             

In [ ]:
# function to clean text

def clean_text(data, stopword):
    
    # casefolding
    data = [i.lower() for i in data]
    data = [' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|\d+", " ", i).split()) for i in data]
    res = ' '.join(data) 

    # tokenizing 
    word_tokens = word_tokenize(res)    
    res = ' '.join([i for i in word_tokens if not i in stopword])
    
    return res





In [ ]:
comment_filtered = []
for i in df_rev['comments']:
    comment_filtered.append(clean_text([i], stop_words))

In [ ]:
#store filtered comments
df_rev['comments_cleaned'] = comment_filtered

In [ ]:
df_rev

,review_id,createdAt,localizedDate,reviewee_id,recommendedNumberOfLines,roomTypeListingTitle,language,highlightReviewMentioned,localizedReview,highlightedReviewSentence,response,reviewer_id,rating,comments,listing_id,auto_posting,comments_cleaned
4407959,701795307,2020-10-18T20:29:21Z,October 2020,26167989,None,None,en,None,None,None,None,35407023,5,Great location! Apartment was exactly what I ...,33586792,no,great location apartment exactly expected davi...
4159841,650225123,2020-08-12T20:30:02Z,August 2020,135568005,None,None,en,None,None,None,None,56070405,5,Stacy's place is a perfect place for the singl...,19347379,no,stacy place perfect place single traveler look...
1541138,280218481,2018-06-22T21:19:28Z,June 2018,55214361,None,None,en,None,None,None,None,105351673,5,Great location. Fabulous place. Bed is super c...,10687245,no,great location fabulous place bed super comfor...
4814331,731728095,2021-02-15T21:59:44Z,February 2021,227777297,None,None,en,None,None,None,None,48680206,5,"Hawk has a Great place, and he's a courteous r...",44614072,no,hawk great place courteous responsive host pla...
6063231,444917120796500217,2021-09-05T20:50:53Z,September 2021,109703164,None,None,en,None,None,None,None,368951484,5,"Wonderful place, close to anywhere need to get...",16617931,no,wonderful place close anywhere need get host r...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4589953,714731679,2020-12-11T20:34:49Z,December 2020,3216904,None,None,en,None,None,None,None,245342304,5,I'm a regular Airbnb traveler and would rate D...,46732672,no,regular airbnb traveler would rate dave katie ...
3849592,621848771,2020-04-05T20:04:37Z,April 2020,122873018,None,None,en,None,None,None,None,33079673,5,Jayme’s place is in the perfect location. It w...,26037016,no,jayme place perfect location sparking clean ab...
5378571,765721878,2021-06-02T21:28:10Z,June 2021,266355572,None,None,en,None,None,None,None,399260878,5,Very much appreciated the cleanliness and atte...,43076943,no,much appreciated cleanliness attention detail ...
3828815,619713604,2020-03-18T21:16:35Z,March 2020,295242098,None,None,en,None,None,None,None,244828248,5,This house was clean and spacious. It was clos...,38646033,no,house clean spacious close main attractions sa...


In [ ]:
for i in df_rev.columns:
    if i == 'listing_id' or i == 'id' or i == 'reviewer_id':
        df_rev[i] = df_rev[i].astype(np.object)
    elif i == 'date' :
        df_rev[i] = pd.to_datetime(df_rev[i])
    else : 
        pass

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:

def pos_tagger(nltk_tag):
  """allowing some figures of speech"""
    if nltk_tag.startswith('J'):
        return wordnet.ADJ
    elif nltk_tag.startswith('V'):
        return wordnet.VERB
    elif nltk_tag.startswith('N'):
        return wordnet.NOUN
    elif nltk_tag.startswith('R'):
        return wordnet.ADV
    else:         
        return None
 

def lemma_nlk(sentence):
  """function to lemmatize"""
  pos_tagged = nltk.pos_tag(sentence)
  wordnet_tagged = list(map(lambda x: (x[0], pos_tagger(x[1])), pos_tagged))
  lemmatized_sentence = []
  for word, tag in wordnet_tagged:
    if tag is None:
      # if there is no available tag, append the token as is
      lemmatized_sentence.append(word)
    else:       
      # else use the tag to lemmatize the token
      lemmatized_sentence.append(lemmatizer.lemmatize(word, tag))
  lemmatized_sentence = " ".join(lemmatized_sentence)
  return lemmatized_sentence


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [ ]:
nltk.download('omw-1.4')
comment_lemma = []
for i in df_rev['comments_cleaned']:
  comment_lemma.append(lemma_nlk(i.split(' ')))

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [ ]:
df_rev['comments_lemmatized'] = comment_lemma

In [ ]:
df_rev.to_csv('/content/sample_data/reviews_cleaned.csv')

In [ ]:
df_rev.head(5)

,review_id,createdAt,localizedDate,reviewee_id,recommendedNumberOfLines,roomTypeListingTitle,language,highlightReviewMentioned,localizedReview,highlightedReviewSentence,response,reviewer_id,rating,comments,listing_id,auto_posting,comments_cleaned,comments_lemmatized
4407959,701795307,2020-10-18T20:29:21Z,October 2020,26167989,None,None,en,None,None,None,None,35407023,5,Great location! Apartment was exactly what I ...,33586792,no,great location apartment exactly expected davi...,great location apartment exactly expect david ...
4159841,650225123,2020-08-12T20:30:02Z,August 2020,135568005,None,None,en,None,None,None,None,56070405,5,Stacy's place is a perfect place for the singl...,19347379,no,stacy place perfect place single traveler look...,stacy place perfect place single traveler look...
1541138,280218481,2018-06-22T21:19:28Z,June 2018,55214361,None,None,en,None,None,None,None,105351673,5,Great location. Fabulous place. Bed is super c...,10687245,no,great location fabulous place bed super comfor...,great location fabulous place bed super comfor...
4814331,731728095,2021-02-15T21:59:44Z,February 2021,227777297,None,None,en,None,None,None,None,48680206,5,"Hawk has a Great place, and he's a courteous r...",44614072,no,hawk great place courteous responsive host pla...,hawk great place courteous responsive host pla...
6063231,444917120796500217,2021-09-05T20:50:53Z,September 2021,109703164,None,None,en,None,None,None,None,368951484,5,"Wonderful place, close to anywhere need to get...",16617931,no,wonderful place close anywhere need get host r...,wonderful place close anywhere need get host r...


<a id='sal'></a>
## Section III: Sentiment Analysis

The sentiment using VADER ( Valence Aware Dictionary for Sentiment Reasoning). It is basically a model used for text sentiment analysis that is sensitive to both polarity (positive/negative) and intensity (strength) of emotion. VADER relies on a dictionary that maps lexical features to emotion intensities known as sentiment scores. The sentiment score of a text can be obtained by summing up the intensity of each word in the text.

In [ ]:
def sentiment(comp):
    if comp >= 0.05:
        return 'positive'
    elif (comp > -0.05) and (comp < 0.05):
        return 'neutral'
    elif comp <= -0.05 :
        return 'negative'

In [ ]:
# initialize sentiment analyzer

analyzer = SentimentIntensityAnalyzer()
# calculate compound score

compound_score = []
for i in df_rev['comments_lemmatized']:
    compound_score.append(analyzer.polarity_scores(i)['compound'])

In [ ]:
df_rev['compound_score'] = compound_score

In [ ]:
sentiment(df_rev['compound_score'].values[1])

'positive'

In [ ]:
sent = []
for i in range(0, len(df_rev)):
  sent.append(sentiment(df_rev['compound_score'].values[i]))

In [ ]:
# create new feature to store sentiment

df_rev['sentiment'] = sent

In [ ]:
df_rev['sentiment'].value_counts()

positive    999171
neutral      16163
negative      8666
Name: sentiment, dtype: int64

In [ ]:
df_rev = df_rev[df_rev['sentiment']=='negative']

In [ ]:
df_rev.shape

(8666, 20)

In [ ]:
df_rev.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8666 entries, 6914574 to 7275325
Data columns (total 20 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   review_id                  8666 non-null   int64  
 1   createdAt                  8666 non-null   object 
 2   localizedDate              8666 non-null   object 
 3   reviewee_id                8666 non-null   int64  
 4   recommendedNumberOfLines   8666 non-null   object 
 5   roomTypeListingTitle       8666 non-null   object 
 6   language                   8666 non-null   object 
 7   highlightReviewMentioned   8666 non-null   object 
 8   localizedReview            8666 non-null   object 
 9   highlightedReviewSentence  8666 non-null   object 
 10  response                   8666 non-null   object 
 11  reviewer_id                8666 non-null   object 
 12  rating                     8666 non-null   int64  
 13  comments                   8666 non-nul

In [ ]:
df_us.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5460445 entries, 0 to 5697209
Data columns (total 14 columns):
 #   Column                Dtype  
---  ------                -----  
 0   user_id               int64  
 1   first_name            object 
 2   is_superhost          object 
 3   recommendation_count  float64
 4   has_profile_pic       object 
 5   total_listings_count  float64
 6   location              object 
 7   reviewee_count        float64
 8   verification_labels   object 
 9   listings_count        float64
 10  created_at            object 
 11  identity_verified     object 
 12  picture_url           object 
 13  picture_url_status    bool   
dtypes: bool(1), float64(4), int64(1), object(8)
memory usage: 588.4+ MB


In [ ]:
df_rev.rename(columns = {'reviewer_id':'user_id'}, inplace = True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5047: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [ ]:
df_sent_neg = pd.merge(df_rev,df_us,on = 'user_id')

In [ ]:
df_sent_neg.head()

,review_id,createdAt,localizedDate,reviewee_id,recommendedNumberOfLines,roomTypeListingTitle,language,highlightReviewMentioned,localizedReview,highlightedReviewSentence,...,has_profile_pic,total_listings_count,location,reviewee_count,verification_labels,listings_count,created_at,identity_verified,picture_url,picture_url_status
0,527547437718574795,2021-12-28T21:02:35Z,December 2021,47848971,None,None,en,None,None,None,...,True,0.0,"St Pauls, New South Wales, Australia",37.0,"['Email address', 'Phone number']",0.0,2019-01-26T17:05:17Z,True,https://a0.muscache.com/im/pictures/user/6dad0...,True
1,639521231,2020-07-18T19:57:43Z,July 2020,3364114,None,None,en,None,None,None,...,True,0.0,"New York, New York, United States",17.0,"['Email address', 'Phone number', 'Reviewed', ...",0.0,2014-11-15T20:08:02Z,True,https://a0.muscache.com/im/pictures/user/11b01...,True
2,745390655,2021-04-04T17:37:36Z,April 2021,124463434,None,None,en,None,None,None,...,True,0.0,"Oakdale, Louisiana, United States",24.0,"['Phone number', 'Government ID', 'Government ...",0.0,2018-09-15T18:16:48Z,True,https://a0.muscache.com/im/pictures/user/da765...,True
3,461037462,2019-05-30T22:11:19Z,May 2019,101537031,None,None,en,None,None,None,...,True,0.0,"Reno, Nevada, United States",6.0,"['Email address', 'Phone number']",0.0,2019-04-18T18:55:12Z,True,https://a0.muscache.com/im/pictures/user/c9521...,True
4,394090022824190114,2021-06-27T17:46:31Z,June 2021,28695156,None,None,en,None,None,None,...,True,0.0,"Gahanna, Ohio, United States",0.0,['Phone number'],0.0,2016-07-13T11:29:51Z,True,https://a0.muscache.com/im/pictures/user/7adca...,True


In [ ]:
df_sent_neg.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8176 entries, 0 to 8175
Data columns (total 33 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   review_id                  8176 non-null   int64  
 1   createdAt                  8176 non-null   object 
 2   localizedDate              8176 non-null   object 
 3   reviewee_id                8176 non-null   int64  
 4   recommendedNumberOfLines   8176 non-null   object 
 5   roomTypeListingTitle       8176 non-null   object 
 6   language                   8176 non-null   object 
 7   highlightReviewMentioned   8176 non-null   object 
 8   localizedReview            8176 non-null   object 
 9   highlightedReviewSentence  8176 non-null   object 
 10  response                   8176 non-null   object 
 11  user_id                    8176 non-null   object 
 12  rating                     8176 non-null   int64  
 13  comments                   8176 non-null   objec

In [ ]:
df_sent_neg.to_csv('/content/sample_data/sentiment_analysis_negative.csv')

<a id='pred'></a>
## Section IV: Predictive Analysis

The dataset seems incomplete for analysis, using a pretrained Model, the Age, Gender and Race of the users will be prediicted.

The Deepface Model was used and the predictions was added to the dataset

In [ ]:
! pip install deepface

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 65 kB 2.1 MB/s 
     |████████████████████████████████| 87 kB 5.9 MB/s 
     |████████████████████████████████| 2.3 MB 45.3 MB/s 
  Created wheel for fire: filename=fire-0.4.0-py2.py3-none-any.whl size=115942 sha256=80873021fd8b26413223524d0620d87bc1a8d953fd37e7f90a280dd66775aae1
  Stored in directory: /root/.cache/pip/wheels/8a/67/fb/2e8a12fa16661b9d5af1f654bd199366799740a85c64981226
Successfully built fire


In [ ]:
import cv2
import matplotlib.pyplot as plt
from deepface import DeepFace

Directory  /root /.deepface created
Directory  /root /.deepface/weights created


In [ ]:

from PIL import Image
from PIL import UnidentifiedImageError
import requests
from io import BytesIO
def read_image(url):
  #url = 'https://a0.muscache.com/im/users/1/profile_pic/1266266786/original.jpg?aki_policy=profile_x_medium'
  response = requests.get(url)
  try:
    img = Image.open(BytesIO(response.content)) # open the image file
    return img
  except UnidentifiedImageError:
    return None


In [ ]:
Age =[]
Race= []
Gender = []


for i in df_sent_neg.picture_url.values:
  img = read_image(i)
  if img == None:
    Age.append('nan')
    Race.append('nan')
    Gender.append('nan')
  else:
    img.convert('RGB').save('img.jpg')
    img1 = cv2.imread('img.jpg')
    result = DeepFace.analyze(img1,actions=['age', 'gender', 'race',],enforce_detection =False)
    #plt.imshow(img1[:,:,::-1])
    #plt.show()
    print('Age: ', result['age'],'Race: ',result['dominant_race'],'Gender: ',result['gender'])
    print(np.where(df_sent_neg.picture_url.values==i),'out of', len(df_sent_neg.picture_url.values))

    Age.append(result['age'])
    Race.append(result['dominant_race'])
    Gender.append(result['gender'])



Action: race: 100%|██████████| 3/3 [00:00<00:00,  7.88it/s]


Age:  29 Race:  white Gender:  Man
(array([0]),) out of 8176


Action: race: 100%|██████████| 3/3 [00:00<00:00,  8.62it/s]


Age:  28 Race:  white Gender:  Man
(array([1]),) out of 8176


Action: race: 100%|██████████| 3/3 [00:00<00:00,  9.12it/s]


...

Age:  23 Race:  black Gender:  Man
(array([8162]),) out of 8176


Action: race: 100%|██████████| 3/3 [00:00<00:00,  6.70it/s]


Age:  35 Race:  white Gender:  Man
(array([8163]),) out of 8176


Action: race: 100%|██████████| 3/3 [00:00<00:00,  6.63it/s]


Age:  25 Race:  middle eastern Gender:  Man
(array([8164]),) out of 8176


In [ ]:
df_sent_neg['Age']= Age
df_sent_neg['Gender']= Gender
df_sent_neg['Race']= Race


In [ ]:
df_sent_neg.to_csv('/content/drive/MyDrive/Datasets/data_predicted_ml.csv')

##  REFERENCES


1. https://huggingface.co/blog/sentiment-analysis-python
2. https://github.com/mohanrishabh/Sentiment-analysis--Airbnb-data/blob/master/3. Sensitivity_Analysis.ipynb
4. https://github.com/CahyaPutera/ASHEVILLE-AIRBNB-ANALYSIS
5. https://towardsdatascience.com/data-sampling-methods-in-python-a4400628ea1b
6. https://towardsdatascience.com/   the-most-favorable-pre-trained-sentiment-classifiers-in-python-9107c06442c6
7. https://towardsdatascience.com/detecting-bad-customer-reviews-with-nlp-d8b36134dc7e 
8. https://stackabuse.com/python-for-nlp-tokenization-stemming-and-lemmatization-with-spacy-library/
9. https://www.machinelearningplus.com/nlp/lemmatization-examples-python/
10. https://neptune.ai/blog/tokenization-in-nlp
11. https://realpython.com/pandas-merge-join-and-concat/
12. https://www.analyticsvidhya.com/blog/2021/06/sentiment-analysis-using-nltk-a-practical-approach/
13. https://ashleygingeleski.com/2021/03/31/sentiment-analysis-of-product-reviews-with-python-using-nltk/
14. https://medium.com/@joel_34096/sentiment-analysis-of-movie-reviews-in-nltk-python-4af4b76a6f3
15. https://www.analyticsvidhya.com/blog/2021/08/text-preprocessing-techniques-for-performing-sentiment-analysis/
https://www.kaggle.com/amitkumardas/sentiment-train 

